# Geo QA

In [3]:
import os
import pandas as pd
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

In [4]:
load_dotenv()

True

In [70]:
def get_URIs(rdf_file, prefix):
    """
    Retrieve URIs from an rdf file
    """

    from rdflib import Graph

    try:
        uris = []
        g = Graph() # Create a Graph
        g.parse(rdf_file) # Parse in an RDF file
        for node in g:
            if str(node[0]).startswith(prefix):
                uris += [str(node[0])]
            else:
                pass
        return list(set(uris)) # remove duplicates
    except:
        print('error: couldn\'t parse file')
        return []

In [71]:
uris = get_URIs('data/rdf/lithology.rdf', 'http://resource.geolba.ac.at/lithology/')
print(len(uris))
uris

258


['http://resource.geolba.ac.at/lithology/251',
 'http://resource.geolba.ac.at/lithology/61',
 'http://resource.geolba.ac.at/lithology/148',
 'http://resource.geolba.ac.at/lithology/229',
 'http://resource.geolba.ac.at/lithology/252',
 'http://resource.geolba.ac.at/lithology/225',
 'http://resource.geolba.ac.at/lithology/232',
 'http://resource.geolba.ac.at/lithology/97',
 'http://resource.geolba.ac.at/lithology/255',
 'http://resource.geolba.ac.at/lithology/144',
 'http://resource.geolba.ac.at/lithology/59',
 'http://resource.geolba.ac.at/lithology/109',
 'http://resource.geolba.ac.at/lithology/112',
 'http://resource.geolba.ac.at/lithology/187',
 'http://resource.geolba.ac.at/lithology/85',
 'http://resource.geolba.ac.at/lithology/114',
 'http://resource.geolba.ac.at/lithology/173',
 'http://resource.geolba.ac.at/lithology/33',
 'http://resource.geolba.ac.at/lithology/15',
 'http://resource.geolba.ac.at/lithology/158',
 'http://resource.geolba.ac.at/lithology/94',
 'http://resource.ge

In [29]:
# urls = [
#     "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/28&lang=de",
#     "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/83&lang=de",
#     "https://thesaurus.geolba.ac.at/?uri=http://resource.geolba.ac.at/minres/78&lang=de",
#     "http://resource.geolba.ac.at/lithology/35"
# ]

In [73]:
"""
Overriding the load() function of SeleniumURLLoader
"""
from langchain.document_loaders import SeleniumURLLoader

import logging
logger = logging.getLogger(SeleniumURLLoader.__name__)

from typing import TYPE_CHECKING, List, Literal, Optional, Union
from langchain.docstore.document import Document

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

class SeleniumURLLoaderCustom(SeleniumURLLoader):
    def load(self) -> List[Document]:
        """Load the specified URLs using Selenium and create Document instances.

        Returns:
            List[Document]: A list of Document instances with loaded content.
        """
        from unstructured.partition.html import partition_html

        docs: List[Document] = list()
        driver = self._get_driver()

        for url in tqdm(self.urls):
            try:
                driver.get(url)
                element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".abstract"))
                    # EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".abstract"))
                )
                text = element.text
                # elements = partition_html(text=page_content)
                # text = "\n\n".join([str(el) for el in elements])
                metadata = {"source": url}
                docs.append(Document(page_content=text, metadata=metadata))
            except Exception as e:
                if self.continue_on_failure:
                    logger.error(f"Error fetching or processing {url}, exception: {e}")
                else:
                    raise e

        driver.quit()
        return docs

In [74]:
loader = SeleniumURLLoaderCustom(urls=uris, browser='firefox')

docs = loader.load()
docs

 75%|███████▌  | 194/258 [02:02<00:40,  1.59it/s]Error fetching or processing http://resource.geolba.ac.at/lithology/adms/0.1, exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16

 81%|████████  | 208/258 [02:20<00:32,  1.53it/s]Error fetching or processing http://resource.geolba.ac.at/lithology/0, exception: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:183:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:395:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16

100%|██████████| 258/258 [03:01<00:00,  1.42it/s]


[Document(page_content='A collective term for plutonic rocks in which the quartz content is more than 90% of the felsic minerals. Now defined modally in QAPF (Streckeisen, 1973) field 1a (Le Maitre et al., 2005).', metadata={'source': 'http://resource.geolba.ac.at/lithology/251'}),
 Document(page_content='Ultramafic rock composed almost entirely of one or more pyroxenes and occasionally biotite, hornblende, or olivine (Le Maitre et al., 2005).', metadata={'source': 'http://resource.geolba.ac.at/lithology/61'}),
 Document(page_content='A fault rock which is cohesive and characterised by a well-developed schistosity resulting from a tectonic reduction in grain size within a ductile shear-zone, and which commonly contains rounded porphyroclasts and lithic fragments of similar composition to the minerals in the matrix (Fettes & Desmons, 2007; Passchier & Trouw, 1996).', metadata={'source': 'http://resource.geolba.ac.at/lithology/148'}),
 Document(page_content='A poorly sorted to unsortet c

In [75]:
embedding = OpenAIEmbeddings()

In [87]:
# we already have a persisted DB dir, only need to recompute this on document structure changes 
persist_directory = "chroma/openai"
vectordb = Chroma.from_documents(documents=docs[0:10], embedding=embedding, persist_directory=persist_directory)
vectordb.persist()
vectordb = None

In [88]:
persist_directory = "chroma/openai" # we can load the persisted database from disk, and use it as normal


vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

retriever = vectordb.as_retriever(search_kwargs={"k": 8})

llm = OpenAI(model_name="text-davinci-003", 
             temperature=0.9, 
             max_tokens=1000)

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff", 
                                 retriever=retriever,
                                 verbose=True)

In [82]:
prompt = 'are there rocks with quartz? return the source.'

qa.run(prompt)



> Entering new  chain...

> Finished chain.


' Yes, according to Le Maitre et al. (2005), plutonic rocks with a quartz content of more than 90% of the felsic minerals are defined modally in QAPF field 1a.'